In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 训练集、测试集特征设计与数据集构造

In [2]:
import pandas as pd
from fastai.tabular import *
from tqdm import trange
from utils import add_datepart, split_vals

In [3]:
root = Path('../data')
# root.ls()

- 以 `log.csv` 结尾的是处理后的数据
- 以 `log_exposed.csv` 结尾的是标签数据

## 广告及其属性数据

In [4]:
ads_fix = pd.read_csv(root/'ads_fix2.csv', low_memory=False, encoding='utf-8')
ads_fix.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,age,area,behavior,...,device,education,gender,status,work,state,price,time,people,cnt
0,570794,2019-02-18 22:48:48,29117,-1,1,44,1,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,51.0,-999.0,-999.0,1
1,126281,2019-03-01 17:46:14,24436,-1,1,20,30,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,95.0,-999.0,-999.0,1
2,286179,2019-02-17 05:39:44,10055,11192,13,84,34,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,166.0,-999.0,-999.0,1
3,19707,2019-03-15 05:41:57,4427,11822,13,12,34,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,184.0,"17592185978880,17592185978880,17592185978880,1...",-999.0,8
4,19707,2019-03-14 05:38:52,4427,11822,13,12,34,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,170.0,"17592185978880,17592185978880,17592185978880,1...",-999.0,7


In [5]:
ads_fix.columns

Index(['广告id', '创建时间', '广告账户id', '商品id', '商品类型', '广告行业id', '素材尺寸', 'age',
       'area', 'behavior', 'connectionType', 'consuptionAbility', 'device',
       'education', 'gender', 'status', 'work', 'state', 'price', 'time',
       'people', 'cnt'],
      dtype='object')

In [6]:
ads_crt = pd.read_csv(root/'ads_crt.csv', low_memory=False, encoding='utf-8')
ads_crt.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,age,area,behavior,...,device,education,gender,status,work,state,price,time,people,cnt
0,31.0,2019-03-12 23:22:50,12577.0,18683.0,13.0,224.0,40.0,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999.0,"281474976694272,281474976694272,28147497669427...",area:11442,3
1,32.0,2018-11-30 07:40:17,18752.0,32534.0,13.0,136.0,40.0,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,83.0,-999.0,area:7572,3
2,84.0,2019-03-06 23:52:47,789.0,18194.0,13.0,98.0,30.0,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,100.0,-999.0,area:6410,3
3,85.0,2019-02-27 07:52:07,6944.0,11250.0,13.0,170.0,34.0,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999.0,"281474976694272,281474976694272,28147497669427...","age:217,601,202,837,942,638,287,5,394,347,731,...",3
4,128.0,2019-03-06 21:40:09,9895.0,31047.0,13.0,17.0,30.0,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999.0,"281474976694272,281474976694272,28147497669427...",area:4672,3


In [7]:
all(ads_crt.columns == ads_fix.columns)

True

In [8]:
new_ads = pd.concat([ads_fix, ads_crt], axis=0, ignore_index=True)
new_ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,age,area,behavior,...,device,education,gender,status,work,state,price,time,people,cnt
0,570794.0,2019-02-18 22:48:48,29117.0,-1.0,1.0,44.0,1.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,51.0,-999.0,-999.0,1
1,126281.0,2019-03-01 17:46:14,24436.0,-1.0,1.0,20.0,30.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,95.0,-999.0,-999.0,1
2,286179.0,2019-02-17 05:39:44,10055.0,11192.0,13.0,84.0,34.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,166.0,-999.0,-999.0,1
3,19707.0,2019-03-15 05:41:57,4427.0,11822.0,13.0,12.0,34.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,184.0,"17592185978880,17592185978880,17592185978880,1...",-999.0,8
4,19707.0,2019-03-14 05:38:52,4427.0,11822.0,13.0,12.0,34.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,170.0,"17592185978880,17592185978880,17592185978880,1...",-999.0,7


In [9]:
new_ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281927 entries, 0 to 281926
Data columns (total 22 columns):
广告id                 281927 non-null float64
创建时间                 281927 non-null object
广告账户id               281927 non-null float64
商品id                 281927 non-null float64
商品类型                 281927 non-null float64
广告行业id               281927 non-null float64
素材尺寸                 281927 non-null float64
age                  281927 non-null float64
area                 281927 non-null float64
behavior             281927 non-null float64
connectionType       281927 non-null float64
consuptionAbility    281927 non-null float64
device               281927 non-null float64
education            281927 non-null float64
gender               281927 non-null float64
status               281927 non-null float64
work                 281927 non-null float64
state                281927 non-null float64
price                281927 non-null float64
time                 281927 non-nul

- 数据格式需要处理

In [10]:
for i in ['广告id', '广告账户id', '商品id', '商品类型', '广告行业id']:
    new_ads[i] = new_ads[i].astype('int64')

In [11]:
new_ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281927 entries, 0 to 281926
Data columns (total 22 columns):
广告id                 281927 non-null int64
创建时间                 281927 non-null object
广告账户id               281927 non-null int64
商品id                 281927 non-null int64
商品类型                 281927 non-null int64
广告行业id               281927 non-null int64
素材尺寸                 281927 non-null float64
age                  281927 non-null float64
area                 281927 non-null float64
behavior             281927 non-null float64
connectionType       281927 non-null float64
consuptionAbility    281927 non-null float64
device               281927 non-null float64
education            281927 non-null float64
gender               281927 non-null float64
status               281927 non-null float64
work                 281927 non-null float64
state                281927 non-null float64
price                281927 non-null float64
time                 281927 non-null object
p

- 删除不必要的行

- 按创建时间进行排序

In [12]:
new_ads.sort_values('创建时间', inplace=True)
new_ads.reset_index(drop=True, inplace=True)

In [13]:
new_ads['date'] = new_ads['创建时间']

In [14]:
new_ads['date'] = new_ads['date'].apply(lambda x: x[:10])

In [15]:
new_ads.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,age,area,behavior,...,education,gender,status,work,state,price,time,people,cnt,date
0,593323,2015-04-07 09:43:55,26889,-1,3,85,64.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,"281474976710655,281474976710655,28147497671065...",all,3,2015-04-07
1,146563,2015-06-04 09:11:32,23511,-1,3,118,64.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,40.0,-999.0,"age:217,202,837,602,935,942,638,287,5,394,536,...",3,2015-06-04
2,216969,2016-03-10 06:35:34,26970,-1,1,33,1.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,34.0,-999.0,"age:1,347,787,753,601,522,202,229,731,333,393,...",3,2016-03-10
3,542734,2016-03-10 06:35:35,26970,-1,1,33,34.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,39.0,-999.0,"age:1,347,787,753,601,522,202,229,731,333,393,...",3,2016-03-10
4,357401,2016-03-10 06:35:35,26970,-1,1,33,1.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,33.0,-999.0,"age:1,347,787,753,601,522,202,229,731,333,393,...",3,2016-03-10


- 3/19 没有标签  删除

In [16]:
new_ads = new_ads.iloc[:-7778, :]

In [17]:
sta_info = new_ads.groupby('date').count()
sta_info.shape
sta_info.head()

(630, 22)

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,age,area,behavior,...,device,education,gender,status,work,state,price,time,people,cnt
date,,,,,,,,,,,,,,,,,,,,,
2015-04-07,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2015-06-04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2016-03-10,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
2016-03-12,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
2016-03-14,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [18]:
sta_info.tail()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,age,area,behavior,...,device,education,gender,status,work,state,price,time,people,cnt
date,,,,,,,,,,,,,,,,,,,,,
2019-03-14,11509,11509,11509,11509,11509,11509,11509,11509,11509,11509,...,11509,11509,11509,11509,11509,11509,11509,11509,11509,11509
2019-03-15,12135,12135,12135,12135,12135,12135,12135,12135,12135,12135,...,12135,12135,12135,12135,12135,12135,12135,12135,12135,12135
2019-03-16,5976,5976,5976,5976,5976,5976,5976,5976,5976,5976,...,5976,5976,5976,5976,5976,5976,5976,5976,5976,5976
2019-03-17,6213,6213,6213,6213,6213,6213,6213,6213,6213,6213,...,6213,6213,6213,6213,6213,6213,6213,6213,6213,6213
2019-03-18,7376,7376,7376,7376,7376,7376,7376,7376,7376,7376,...,7376,7376,7376,7376,7376,7376,7376,7376,7376,7376


- 从 2/16 开始有标签
- 日期 < 2/16 的 

In [19]:
a = new_ads['date'].unique()
a
a.shape

array(['2015-04-07', '2015-06-04', '2016-03-10', '2016-03-12', ..., '2019-03-15', '2019-03-16', '2019-03-17',
       '2019-03-18'], dtype=object)

(630,)

In [20]:
a[-32:]

array(['2019-02-15', '2019-02-16', '2019-02-17', '2019-02-18', '2019-02-19', '2019-02-20', '2019-02-21', '2019-02-22',
       '2019-02-23', '2019-02-24', '2019-02-25', '2019-02-26', '2019-02-27', '2019-02-28', '2019-03-01', '2019-03-02',
       '2019-03-03', '2019-03-04', '2019-03-05', '2019-03-06', '2019-03-07', '2019-03-08', '2019-03-09', '2019-03-10',
       '2019-03-11', '2019-03-12', '2019-03-13', '2019-03-14', '2019-03-15', '2019-03-16', '2019-03-17', '2019-03-18'],
      dtype=object)

In [21]:
new_ads.head(10).loc[:, '广告id':'广告账户id']

,广告id,创建时间,广告账户id
0,593323,2015-04-07 09:43:55,26889
1,146563,2015-06-04 09:11:32,23511
2,216969,2016-03-10 06:35:34,26970
3,542734,2016-03-10 06:35:35,26970
4,357401,2016-03-10 06:35:35,26970
5,561630,2016-03-10 09:35:18,16064
6,74944,2016-03-12 03:57:49,16064
7,100580,2016-03-12 06:42:23,661
8,87171,2016-03-14 07:05:45,25742
9,28141,2016-03-21 09:37:01,28417


In [22]:
new_ads.shape

(274149, 23)

### 解决方案
1. 时间列 排序
2. 统计每种时间的个数
3. 按索引取出对应的时间子表
4. 读取标签并 merge

- 获取时间

In [23]:
t_cnt = sta_info['广告id'].to_list()
t_cnt[:5]

[1, 1, 4, 2, 1]

- 获取个数
- 2/16~3/19

In [24]:
a[:-32]

array(['2015-04-07', '2015-06-04', '2016-03-10', '2016-03-12', ..., '2019-02-11', '2019-02-12', '2019-02-13',
       '2019-02-14'], dtype=object)

In [25]:
signal = sta_info[-32:]
sum(signal.cnt.to_list())

262128

- 最后的 262128 是从 2/15 开始的，前面的是之前创建的广告

In [26]:
last = new_ads.iloc[-262128:, :]
first = new_ads.iloc[:-262128, :]

- 对first中的和每天标签 静态广告
- 对last的和后一天 标签  动态广告
- 假设上述成立

- 取子表，读标签，合并

In [27]:
l_date = list(a[-31:])
l_date.append('2019-03-19')

- 新表格

In [28]:
train = pd.DataFrame(columns=new_ads.columns.to_list() + ['曝光量'])
train.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,age,area,behavior,...,gender,status,work,state,price,time,people,cnt,date,曝光量


In [29]:
for lf in trange(32):
    label = pd.read_csv(f'../data/{l_date[lf]}_log_exposed.csv', encoding='utf-8')
    # 复制 避免混淆
    tmp = first.copy()
    tmp1 = pd.merge(tmp, label, how='inner', left_on='广告id', right_on='曝光广告id')
    train = pd.concat([train, tmp1])

  0%|          | 0/32 [00:00<?, ?it/s]/home/lab/Softwares/miniconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
100%|██████████| 32/32 [00:01<00:00, 28.84it/s]


In [30]:
train.shape

(92709, 25)

- last 部分的标签

In [31]:
lens = signal.广告id.to_list()
lens[:5]

[224, 377, 822, 6389, 3917]

In [32]:
last.reset_index(drop=True, inplace=True)
last.head()

,广告id,创建时间,广告账户id,商品id,商品类型,广告行业id,素材尺寸,age,area,behavior,...,education,gender,status,work,state,price,time,people,cnt,date
0,602287,2019-02-15 01:26:20,28899,-1,1,136,64.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,58.0,"8796093021184,8796093021184,8796093021184,8796...",-999.0,3,2019-02-15
1,543961,2019-02-15 01:31:57,7715,12415,13,169,1.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,150.0,-999.0,area:7572,3,2019-02-15
2,598361,2019-02-15 01:32:00,7715,12415,13,84,34.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,150.0,-999.0,area:7572,3,2019-02-15
3,130267,2019-02-15 01:42:01,13804,-1,1,242,40.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,"4398046494720,4398046494720,4398046494720,4398...","age:217,837,942,638,287,5,394,731,739,393,366,...",3,2019-02-15
4,575682,2019-02-15 01:52:08,13108,29346,13,170,1.0,-999.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,"281474976645120,281474976645120,28147497664512...","age:217,601,202,837,942,638,287,5,394,347,731,...",3,2019-02-15


In [33]:
la = pd.read_csv(f'../data/2019-02-19_log_exposed.csv', encoding='utf-8')

In [34]:
la.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57275 entries, 0 to 57274
Data columns (total 2 columns):
曝光广告id    57275 non-null int64
曝光量       57275 non-null float64
dtypes: float64(1), int64(1)
memory usage: 895.0 KB


In [35]:
idx, l1 = 0, len(last)

In [36]:
for lf in trange(32):
    label = pd.read_csv(f'../data/{l_date[lf]}_log_exposed.csv', encoding='utf-8')
    # 对应日期的特征
    idx1 = idx+lens[lf]
    tmp = last.iloc[idx:idx1, :]
    idx = idx1
    tmp1 = pd.merge(tmp, label, how='inner', left_on='广告id', right_on='曝光广告id')
    train = pd.concat([train, tmp1])

  0%|          | 0/32 [00:00<?, ?it/s]/home/lab/Softwares/miniconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
100%|██████████| 32/32 [00:02<00:00, 11.83it/s]


In [37]:
train.shape

(210855, 25)

### 时间特征

In [38]:
train['创建时间'] = pd.to_datetime(train['创建时间'])

In [39]:
add_datepart(train, '创建时间', drop=True, time=True)
train.drop(columns='date', inplace=True)

### 人群定向分解

In [40]:
cols = ['age','area','behavior','connectionType','consuptionAbility','device','education','gender','status','work']
func = partial(split_vals, cols=cols, field='people')

In [41]:
x = train.people.apply(lambda x: x.split('|')[0])

In [42]:
y = x.apply(lambda x: x.split(':')[0])

In [43]:
train.head()

,age,area,behavior,cnt,connectionType,consuptionAbility,device,education,gender,people,...,crt_dateIs_month_end,crt_dateIs_month_start,crt_dateIs_quarter_end,crt_dateIs_quarter_start,crt_dateIs_year_end,crt_dateIs_year_start,crt_dateHour,crt_dateMinute,crt_dateSecond,crt_dateElapsed
0,-999.0,-999.0,-999.0,3,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,False,False,False,False,False,False,6,14,44,1460528084
1,-999.0,-999.0,-999.0,3,-999.0,-999.0,-999.0,-999.0,-999.0,os:2,...,False,False,False,False,False,False,7,25,17,1464161117
2,-999.0,-999.0,-999.0,3,-999.0,-999.0,-999.0,-999.0,-999.0,all,...,False,True,False,False,False,False,2,35,46,1472697346
3,-999.0,-999.0,-999.0,3,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,False,False,False,False,False,False,7,54,44,1478678084
4,-999.0,-999.0,-999.0,3,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,...,False,False,False,False,False,False,2,39,19,1480387159


In [44]:
train['is_all_field'] = train['people'] == 'all'
train['has_product_id'] = train['商品id'] != -1  # - 1 表示的无商品 id

In [45]:
train = train.apply(func, axis=1)
train.drop(columns='people', inplace=True)

### fin 训练集

In [46]:
train.columns

Index(['age', 'area', 'behavior', 'cnt', 'connectionType', 'consuptionAbility',
       'device', 'education', 'gender', 'price', 'state', 'status', 'time',
       'work', '商品id', '商品类型', '广告id', '广告行业id', '广告账户id', '曝光广告id', '曝光量',
       '素材尺寸', 'crt_dateYear', 'crt_dateMonth', 'crt_dateWeek', 'crt_dateDay',
       'crt_dateDayofweek', 'crt_dateDayofyear', 'crt_dateIs_month_end',
       'crt_dateIs_month_start', 'crt_dateIs_quarter_end',
       'crt_dateIs_quarter_start', 'crt_dateIs_year_end',
       'crt_dateIs_year_start', 'crt_dateHour', 'crt_dateMinute',
       'crt_dateSecond', 'crt_dateElapsed', 'is_all_field', 'has_product_id'],
      dtype='object')

In [47]:
train.shape

(210855, 40)

### 测试集做相同处理

In [48]:
train.to_csv('../data/train_set.csv', index=None, encoding='utf-8')

### fin 数据集创建